In [1]:
import cv2
import csv
import mediapipe as mp
import cvzone
import time

In [2]:
# Initialize mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.8)  # Limiting to one hand and setting detection confidence
mp_drawing = mp.solutions.drawing_utils  # Utility functions for drawing landmarks

# Define a class to represent multiple-choice questions
class MCQ:
    def __init__(self, data):
        # Initialize question attributes from data
        self.question = data[0]  # First element is the question text
        self.choices = data[1:5]  # Next four elements are the choices
        self.answer = int(float(data[5]))  # Convert answer from string to integer
        self.userAns = None  # Initialize user's answer as None

    # Method to update user's answer based on cursor position
    def update(self, cursor, bboxs):
        for i, bbox in enumerate(bboxs):
            x1, y1, x2, y2 = bbox
            # If cursor is within bounding box of a choice, update user's answer
            if x1 < cursor[0] < x2 and y1 < cursor[1] < y2:
                self.userAns = i + 1  # Index of choice (starting from 1)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0))  # Highlight selected choice

# Read the CSV file and create MCQ objects
mcqList = []
with open('Mcqs.csv', newline='\n') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        mcqList.append(MCQ(row))  # Create MCQ objects from CSV data

# Initialize webcam
cap = cv2.VideoCapture(0)

qNo = 0  # Initialize question number
qTotal = len(mcqList)  # Get total number of questions

# Main loop for real-time processing
while True:
    ret, img = cap.read()  # Read frame from webcam
    img = cv2.flip(img, 1)  # Flip frame horizontally
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert frame to RGB for Mediapipe processing
    results = hands.process(imgRGB)  # Process frame to detect hands
    
    # If hands are detected
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get landmarks for index and middle fingers
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            
            # Convert landmarks to pixel coordinates
            h, w, c = img.shape
            index_tip = (int(index_finger_tip.x * w), int(index_finger_tip.y * h))
            middle_tip = (int(middle_finger_tip.x * w), int(middle_finger_tip.y * h))
            
            # Calculate cursor position between fingers
            cursor = ((index_tip[0] + middle_tip[0]) // 2, (index_tip[1] + middle_tip[1]) // 2)
            length = ((index_tip[0] - middle_tip[0]) ** 2 + (index_tip[1] - middle_tip[1]) ** 2) ** 0.5

            # Draw fingertip indicators and cursor line
            cv2.circle(img, index_tip, 10, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, middle_tip, 10, (255, 0, 255), cv2.FILLED)
            cv2.line(img, index_tip, middle_tip, (255, 0, 255), 3)
            cv2.circle(img, cursor, 15, (0, 255, 0), cv2.FILLED)

            # Check for selection based on finger distance
            if length < 50:  # Threshold for click gesture
                if qNo < qTotal:  # If there are still questions remaining
                    mcq = mcqList[qNo]  # Get current question
                    bboxs = [(25, 100, 140, 180), (270, 100, 385, 180), (25, 250, 140, 330), (270, 250, 385, 330)]  # Adjusted bounding box coordinates
                    mcq.update(cursor, bboxs)  # Update user's answer based on cursor position
                    if mcq.userAns is not None:  # If user has selected an answer
                        time.sleep(1)  # Add delay
                        qNo += 1  # Move to next question

    # Draw question and choices on the frame
    if qNo < qTotal:  # If there are still questions remaining
        mcq = mcqList[qNo]  # Get current question
        # Draw question and choices on the frame
        img, bbox = cvzone.putTextRect(img, mcq.question, [25, 50], 2, 2, offset=30, border=5)
        img, bbox1 = cvzone.putTextRect(img, mcq.choices[0], [50, 150], 2, 2, offset=30, border=5)
        img, bbox2 = cvzone.putTextRect(img, mcq.choices[1], [300, 150], 2, 2, offset=30, border=5)
        img, bbox3 = cvzone.putTextRect(img, mcq.choices[2], [50, 300], 2, 2, offset=30, border=5)
        img, bbox4 = cvzone.putTextRect(img, mcq.choices[3], [300, 300], 2, 2, offset=30, border=5)

    else:  # If all questions have been answered
        # Calculate score
        score = 0
        for mcq in mcqList:
            if mcq.answer == mcq.userAns:
                score += 1
        score = round((score / qTotal) * 100, 2)  # Calculate percentage score
        # Display completion message and score
        img, _ = cvzone.putTextRect(img, "Quiz Completed", [220, 250], 2, 2, offset=30, border=5)
        img, _ = cvzone.putTextRect(img, f'Your Score: {score}%', [200, 400], 2, 2, offset=30, border=5)

    # Resize frame for better display
    resized_img = cv2.resize(img, (800, 600))

    # Display resized frame
    cv2.imshow("Image", resized_img)
    
    # Check for key press to exit
    if cv2.waitKey(5) & 0xFF == 27:
        break

# Release webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()